# Tasks
Organizing the COCO data:

- Create a class that organizes all of the COCO data. It might store the following
    - All the image IDs
    - All the caption IDs
    - Various mappings between image/caption IDs, and associating caption-IDs with captions
        - `image-ID -> [cap-ID-1, cap-ID-2, ...]`
        - `caption-ID -> image-ID`
        - `caption-ID -> caption (e.g. 24 -> "two dogs on the grass")`

# Loading Data

In [2]:
from cogworks_data.language import get_data_path

from pathlib import Path
import json

# load COCO metadata
filename = get_data_path("captions_train2014.json")
with Path(filename).open() as f:
    coco_data = json.load(f)

# >>> len(coco_data["images"])  # number of images
# 82783

# >>> coco_data["images"][0]
# {'license': 5,
#  'file_name': 'COCO_train2014_000000057870.jpg',
#  'coco_url': 'http://images.cocodataset.org/train2014/COCO_train2014_000000057870.jpg',
#  'height': 480,
#  'width': 640,
#  'date_captured': '2013-11-14 16:28:13',
#  'flickr_url': 'http://farm4.staticflickr.com/3153/2970773875_164f0c0b83_z.jpg',
#  'id': 57870}

In [3]:
from gensim.models import KeyedVectors
filename = "glove.6B.200d.txt.w2v"

# this takes a while to load -- keep this in mind when designing your capstone project
glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False)

In [4]:
import pickle
with Path(get_data_path('resnet18_features.pkl')).open('rb') as f:
    resnet18_features = pickle.load(f)

# COCO Data Class

In [13]:
c = coco_data["annotations"]
# print (c[:10])

i = coco_data["images"]
print(type(i))

print(type(coco_data))

<class 'list'>
<class 'dict'>


In [ ]:
class CocoDataManager:
    """
    Attributes
    ----------
    images : list
        

    Methods
    -------
    colorspace(c='rgb')
        Represent the photo in the given colorspace.
    gamma(n=1.0)
        Change the photo's gamma exposure.

    """
    
    def __init__ (self, coco_data):
        """
        Initialize a CocoDataManager class instance

        Parameters
        ----------
        coco_data : dictionary
            The loaded json file as a dictionary
        """
        
        self.images = coco_data["images"]
        self.captions = coco_data["annotations"]
        
        self.imageID_to_captionID = {}
        self.captionID_to_imgID = {}
        
        self.captionID_to_caption = {}
        self.caption_to_captionID = {}
        
        self.imageID_to_url = {}
        
        self.imageIDs = [i["id"] for i in self.images]
        
        for c in self.captions:
            cap_id = c["id"]
            img_id = c["image_id"]
            cap = c["caption"]
            
            # adding data to the imageID to captionID dictionary
            if (img_ID in self.imageID_to_captionID):
                self.imageID_to_captionID[img_id].append(cap_id)
            else:
                self.imageID_to_captionID[img_id] = [cap_id]
            
            # adding data to the captionID to imageID dictionary
            self.captionID_to_imgID[cap_id] = img_id
            
            # adding data to the captionID to caption dictionary
            self.captionID_to_caption[cap_id] = cap
            
        # adding data to the caption to captionID dictionary
        self.caption_to_captionID = {value:key for key, value in captionID_to_caption.items()}
        
        for img in self.images:
            img_id = img["id"]
            img_url = img["coco_url"]
            img_height = img["height"]
            img_width = img["width"]
        
            # adding data to the imageID to url dictionary
            self.imageID_to_url[img_id] = img_url
                
    def getCaptionIDs (self, img_id):
        """
        Gives a list of caption IDs given an image ID

        Parameters
        ----------
        img_id : int
            Id of the image to be accessed

        Returns
        -------
        list
            List of length 4 that contains the caption ids
        """
        return self.imageID_to_captionID[img_id]
    
    def getUrl (self, img_id):
        """
        Gives a coco url given an image ID

        Parameters
        ----------
        img_id : int
            Id of the image to be accessed

        Returns
        -------
        String
            The coco URL used to access/download an image
        """
        return self.imageID_to_url[img_id]